In [ ]:
import requests
from hkvfewspy.utils import untangle
import pandas as pd

In [ ]:
url_locations = 'http://edukovizig.hu/map/eduvizig_allomaspontok.php?vizigkod=1'

In [ ]:
r = requests.get(url_locations)
r.encoding = r.apparent_encoding
u = untangle.parse_raw(r.text)

In [ ]:
d_list = []
for marker in u.markers.marker:
    m = marker
    d = {
        'name':m['nev'], 
        'lat':float(m['lat']), 
        'lon':float(m['lng']), 
        'id':m['voa'], 
        'stream':m['viznev'], 
        'zero_level_mBf':float(m['nullpont']), 
        'threshold_1_cm':float(m['fok1']), 
        'threshold_2_cm':float(m['fok2']), 
        'threshold_3_cm':float(m['fok3']),
        'min_waterlevel_cm':float(m['lkv']),
        'max_waterlevel_cm':float(m['lnv']),    
        'river_km':m['folyamkm'],
        'waterlevel_cm':float(m['vizallas'])
    }
    d_list.append(d)

In [ ]:
df = pd.DataFrame(d_list)
df.head()

In [ ]:
import geopandas

In [ ]:
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))

In [ ]:
gdf.to_file(r'D:\Projects\Pr\2505.12\Eduvizig\Sensor stations (waterstandlocaties)//stations.shp')

In [ ]:
df.tail()

In [ ]:
for idx, id_voa in enumerate(df.id):
    print(id_voa)
    url_loc = 'http://edukovizig.hu/map/export.php?voa={}'.format(id_voa) 
    r = requests.get(url_loc)

    with open("location_id_{}.csv".format(id_voa), "w") as f:
        try:
            r.encoding = r.apparent_encoding            
            f.write(r.text)
        except:
            try:
                r.encoding = 'utf-8'
                f.write(r.text)
            except:
                print('failed: {}'.format(id_voa))
                continue